In [2]:
# Helper packages
import imp
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

# import plotnine
from plotnine import ggplot, aes, geom_density, geom_line, geom_point, ggtitle

import seaborn as sns
# Modeling process
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
import sklearn.metrics 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA



# deep learning functionality
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
from tensorflow.keras import layers


C:\Users\user\AppData\Local\Temp\ipykernel_264\3002871627.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [ ]:
#####lana 

plt.figure(figsize=(13,7))
sns.heatmap(df.corr(),annot=True,vmin=-1,vmax=1)

In [ ]:
# read the dataset
df = pd.read_csv("archive\star_classification.csv")

# encode values for class column
df.replace({'class': {'GALAXY': 0, 'STAR': 1, 'QSO':2}}, inplace=True)

# # remove all columns containing ID at the end
# cleaned = df.drop(df.filter(regex='ID$').columns, axis=1)
# # drop the date column
# cleaned = cleaned.drop(['MJD',"plate"], axis=1)
# cleaned=cleaned.drop(79543)
# # make the X and y varialbes
# X = cleaned.drop('class', axis=1)
# y = cleaned['class']

# # split the dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)

In [ ]:
df

In [ ]:
df.corr().style.background_gradient(cmap="coolwarm")

In [ ]:
df.corr()["class"].sort_values()

In [ ]:
from dis import dis
cleaned = df.drop(['u ','r','i','z','obj_ID','spec_obj_ID','MJD'], axis = 1)

# cleaned = df.drop(['obj_ID','alpha','delta','run_ID','rerun_ID','cam_col','field_ID','fiber_ID'], axis = 1)
# cleaned = df.drop(df.filter(regex='ID$').columns, axis=1)
# drop the date column
# cleaned = cleaned.drop(["u","r","z","alpha","delta","MJD"], axis=1)
cleaned=cleaned.drop(79543)
# make the X and y varialbes
X = cleaned.drop('class', axis=1)
y = cleaned['class']
display(cleaned)


sm = SMOTE(random_state=42)
X, y = sm.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)

In [ ]:
sns.displot(df["run_ID"]);


In [ ]:
sns.displot(df["field_ID"]);


In [ ]:
sns.displot(df["MJD"]);

In [ ]:
cleaned[cleaned["redshift"]>5]

In [ ]:
sns.displot(cleaned["alpha"]);

In [ ]:
sns.displot(cleaned["delta"]);

In [ ]:
sns.displot(x=cleaned["u"]);

In [ ]:
cleaned

In [ ]:
sns.displot(x=cleaned["g"]);

In [ ]:
sns.displot(cleaned["r"]);

In [ ]:
sns.displot(cleaned["i"]);

In [ ]:
sns.displot(cleaned["z"]);

In [ ]:
sns.countplot(x=cleaned["cam_col"]);

In [ ]:
sns.countplot(x=cleaned["class"]);

In [ ]:
sns.displot(cleaned["redshift"]);

In [ ]:
sns.displot(cleaned["plate"]);

# pre-processing

In [ ]:
# Normalizing approach
yj = PowerTransformer(method="yeo-johnson")
scaler = StandardScaler()
# nzv_encoder = VarianceThreshold(threshold=0.1)
# pca = PCA(n_components=7)
# Normalize all numeric features
preprocessor = ColumnTransformer([("norm", yj, selector(dtype_include="number")),
                ("std_encode", scaler, selector(dtype_include="number")),
                # ("nzv_encoder", nzv_encoder, selector(dtype_include="number")),
                # ("pca_encode", pca, selector(dtype_include="number"))
                ])

# Random Forest Classifier

In [ ]:
r_forest = RandomForestClassifier()
model_pipeline = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("knn", r_forest),
])
model_pipeline.fit(X_train,y_train)
predicted = model_pipeline.predict(X_test)
score = model_pipeline.score(X_test,y_test)
model_pipeline_score = np.mean(score)
model_pipeline_score
0.983

In [ ]:
print(classification_report(y_test, predicted))


# SVM

In [ ]:
svm_clf = svm.SVC(kernel='rbf', C=32, random_state=0)
model_pipeline = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("knn", svm_clf),
])
model_pipeline.fit(X_train,y_train)
predicted = model_pipeline.predict(X_test)
score = model_pipeline.score(X_test,y_test)
model_pipeline_score = np.mean(score)
model_pipeline_score
#0.977

In [ ]:
print(classification_report(y_test, predicted)) 

# Logistic Regression

In [ ]:
log_reg=LogisticRegression()
model_pipeline = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("log_reg", log_reg),
])
model_pipeline.fit(X_train,y_train)
predicted = model_pipeline.predict(X_test)
score = model_pipeline.score(X_test,y_test)
model_pipeline_score = np.mean(score)
model_pipeline_score

In [ ]:
print(classification_report(y_test, predicted)) 

In [ ]:
# define loss function
loss = 'accuracy'

# create 10 fold CV object
kfold = KFold(n_splits=5, random_state=123, shuffle=True)

# fit model with 10-fold CV
# results = cross_val_score(model_pipeline, X_train, y_train, cv=kfold, scoring=loss)
# results

In [ ]:
results.mean()

In [ ]:
# Create grid of hyperparameter values
hyper_grid = {'log_reg__max_iter': range(100, 1000,100)}

# Tune a knn model using grid search
grid_search = GridSearchCV(model_pipeline, hyper_grid, cv=kfold, scoring=loss)
results = grid_search.fit(X_train, y_train)


In [ ]:
results.best_params_

# K-Nearest Neighbors


In [ ]:

knn=KNeighborsClassifier(n_neighbors=3)
model_pipeline = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("knn", knn),
])
model_pipeline.fit(X_train,y_train)
predicted = model_pipeline.predict(X_test)
score = model_pipeline.score(X_test,y_test)
model_pipeline_score = np.mean(score)
model_pipeline_score


In [ ]:
print(classification_report(y_test, predicted)) 

# deep learining

In [ ]:
from dis import dis

# cleaned = df.drop(['obj_ID','alpha','delta','run_ID','rerun_ID','cam_col','field_ID','fiber_ID'], axis = 1)
cleaned = df.drop(['u','r','i','z','obj_ID','spec_obj_ID','MJD'], axis = 1)

# cleaned = df.drop(df.filter(regex='ID$').columns, axis=1)
# drop the date column
# cleaned = cleaned.drop(["MJD","plate","cam_col"], axis=1)
cleaned=cleaned.drop(79543)
# make the X and y varialbes
X = cleaned.drop('class', axis=1)
y = cleaned['class']
display(cleaned)


sm = SMOTE(random_state=42)
X, y = sm.fit_resample(X, y)


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)

from tensorflow.keras import utils
# y = utils.to_categorical(y)
# y_train = utils.to_categorical(y_train)
# y_test = utils.to_categorical(y_test)


In [ ]:

# define the keras model
model = Sequential()
model.add(Dense(units=64, input_dim=20, activation="tanh"))
model.add(Dense(units=64,  activation="tanh"))
model.add(Dense(units=32,  activation="tanh"))
model.add(Dense(units=3, activation='softmax'))

# compile the keras model
model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer="rmsprop",
    metrics='accuracy'
)
# fit the model
# model.fit(X, y, epochs=20, validation_split=0.2)
model_pipeline = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("model", model),
])
m1=model_pipeline.fit(X_train,y_train, model__epochs=20, model__validation_split=0.2,model__batch_size=32,)

In [ ]:
predicted=m1.predict(X_test)
y_classes = predicted.argmax(axis=-1)

# model_pipeline.transform(X_test)
print(classification_report(y_test, y_classes)) 

In [ ]:
# define loss function
loss = 'accuracy'

# create 10 fold CV object
kfold = KFold(n_splits=5, random_state=123, shuffle=True)

# fit model with 10-fold CV
results = cross_val_score(model_pipeline, X_train, y_train, cv=kfold, scoring=loss)
results

In [ ]:
# hyper_grid = {'n_neighbors': range(2, 26)}
# grid_search = GridSearchCV(knn, hyper_grid, cv=kfold, scoring=loss)
# results = grid_search.fit(X_train, y_train)

# k-means clustring | trash

In [ ]:
model = sklearn.cluster.KMeans(n_clusters=3,random_state=123)
model

In [ ]:
galaxy = cleaned[cleaned["class"]==0].drop("class",axis=1)
galaxy_centers = map(lambda a: a/galaxy.shape[0],galaxy.sum())
galaxy_centers= np.array(list(galaxy_centers))
galaxy_centers


STAR = cleaned[cleaned["class"]==1].drop("class",axis=1)
STAR_centers = map(lambda a: a/STAR.shape[0],STAR.sum())
STAR_centers= np.array(list(STAR_centers))
STAR_centers



QSO = cleaned[cleaned["class"]==2].drop("class",axis=1)
QSO_centers = map(lambda a: a/QSO.shape[0],QSO.sum())
QSO_centers= np.array(list(QSO_centers))
QSO_centers



In [ ]:
m1=model.fit([galaxy_centers,STAR_centers,QSO_centers])
m1.labels_

In [ ]:
pred= m1.predict(X_test)
print(classification_report(y_test, pred)) 